In [1]:
#mounting spu google drive
from google.colab import drive
drive.mount('/content/drive')

#changing the working directory
import os
os.chdir("/content/drive/MyDrive/SPU DS Graduate RA - EV Project/EV Market Analysis and Consumer Behavior")

!pwd

Mounted at /content/drive
/content/drive/MyDrive/SPU DS Graduate RA - EV Project/EV Market Analysis and Consumer Behavior


## **Building a Deep Learning Sentiment Analysis Model with YouTube Comments**

we are going to do transfer-learning first and see the perfomance of the model

In [2]:
import pandas as pd